In [1]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
import re
import pandas as pd
import sys
from google.colab import files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
""" Design
Define points(index,x,y)
points distance を行列に纏める(0,1)
ODEをpoints用に変換

拡散項だけ分解する必要あり
拡散項以外を計算後、格納項をneibmxでforする




2つのforより全組み合わせを計算
この際，matrixよりneib or Notneibを判断して，先頭に0or1を付与


points_idx = [index, x, y]
points_crd = [x, y]

"""

'\nDefine points(index,x,y)\npoints distance を行列に纏める(0,1)\nODEをpoints用に変換\n\n拡散項だけ分解する必要あり\n拡散項以外を計算後、格納項をneibmxでforする\n\n\n\n\n2つのforより全組み合わせを計算\nこの際，matrixよりneib or Notneibを判断して，先頭に0or1を付与\n\n\npoints_idx = [index, x, y]\npoints_crd = [x, y]\n\n'

In [8]:
""" Progress
csvからpointsを作成
neib_mxを作成


"""

' Progress\ncsvからpointsを作成\nneib_mxを作成\n\n\n'

In [3]:
"""< csvからpointsの要素を取り出す>

drive内のcsvにpointsのindex,x,yを格納
csvとして読み込み
各行を取り出してpoints_idxに格納
x, yのみをpoints_crdに格納

"""

points_temp_csv= pd.read_csv('drive/My Drive/Colab Notebooks/Diffusion-ish/points_temp.csv')
#print(points_temp_csv)
points_idx = []
for index, row in points_temp_csv.iterrows():

    Points_Param_t = row['index'], row['x'], row['y']
    Points_Param_list = list(Points_Param_t)
    points_idx.append(Points_Param_list)


points_crd = []
for a in range(len(points_idx)):
    abc = points_idx[a]
    abd = abc[1:3]
    points_crd.append(abd)

#print(type(points_idx))
print(points_idx)
print(points_crd)

[[0, 250, 250], [1, 200, 250], [2, 250, 350], [3, 300, 250], [4, 350, 250]]
[[250, 250], [200, 250], [250, 350], [300, 250], [350, 250]]


In [5]:
def get_neib_mx(cutoff, points_crd):

    neib_mx = []
    neib_mx = np.zeros((len(points_crd), len(points_crd)))
    for i, pt1 in enumerate(points_crd):
        for j in range(i+1, len(points_crd)):
            pt2 = np.array(points_crd[j])
            pt1 = np.array(pt1)
            dis = np.linalg.norm(pt2-pt1)
            if dis < cutoff:
                neib_mx[i,j] = 1
                neib_mx[j,i] = 1

    return neib_mx

In [9]:
def initial_field(a):
    zero_wusr = np.zeros(a)
    zero_wusp = np.zeros(a)
    zero_clvr = np.zeros(a)
    zero_clvp = np.zeros(a)
    zero_erf = np.zeros(a)
    zero_ham = np.zeros(a)

    return zero_wusr, zero_wusp, zero_clvr, zero_clvp, zero_erf, zero_ham

In [ ]:
def sol_2d_diffusion(x, y, wusr, wusp, clvr, clvp, erf, ham, pull, ID):

    dt, dx, dy, a, step, result_interval =\
       pull[0], pull[1], pull[2], pull[3], pull[4], pull[5]


    for n in range(step):
        wusr0, wusp0, clvr0, clvp0, erf0, ham0 =\
          wusr.copy(), wusp.copy(), clvr.copy(), clvp.copy(), erf.copy(), ham.copy()

        wusr0, wusp0, clvr0, clvp0, erf0, ham0 =\
           np.floor(wusr0), np.floor(wusp0), np.floor(clvr0), np.floor(clvp0), np.floor(erf0), np.floor(ham0)

        for i in range(0, len(wusr)):

                # b_W -> b_Ww, k0_W -> ki_W, a_C -> a_Cc, b_p -> b_pp, k_p -> k_pp, b_H -> b_Hh, k_H -> k_Hh, k_L -> k_Ll, b_L -> b_Ll, D_W -> D_Ww, D_C -> D_Cc, D_L -> D_Ll

                """
                定常項はiのみ
                D項はjを用いてneib_mxを用いて判定

                """


                wusr[i] = k_Wr * ((k_WL/(1 + (erf0[i] / K_WL)**n_WL)) + (k_WC/(1+(clvp0[i] / K_WC )**n_WC))) * (ki_W + ((wusp0[i] / K_WW)**n_WW / (1+(wusp0[i] / K_WW)**n_WW)))  - b_Wr * wusr0[i]

                wusp[i] = k_Wp * wusr0[i] - b_Ww * wusp0[i]
                    + D_Ww * (wusp0[i+1, j] - 2 * wusp0[i, j] + wusp0[i-1, j])  +  D_Ww * (wusp0[i, j+1] - 2 * wusp0[i, j] + wusp0[i, j-1])

                clvr[i] = (k_Cr / (1+(erf0[i]/K_CL)**n_CL)) * (a_Cc + ((wusp0[i]/K_CW)**n_CW / (1 + (wusp0[i]/K_CW)**n_CW))) * (1/(1+(ham0[i]/K_CH)**n_CH))  - b_Cr * clvr0[i]

                clvp[i] = k_Cp * clvr0[i] - b_Cr * clvp0[i]
                    + D_Cc * (clvp0[i+1, j] - 2 * clvp0[i, j] + clvp0[i-1, j]) + D_Cc * (clvp0[i, j+1] - 2 * clvp0[i, j] + clvp0[i, j-1])

                erf[i] = k_Ll - b_Ll * erf0[i]  +
                    D_Ll * (erf0[i+1, j] - 2 * erf0[i, j] + erf0[i-1, j])  +  D_Ll * (erf0[i, j+1] - 2 * erf0[i, j] + erf0[i, j-1])




                """
                wusr[i] = k_Wr * ((k_WL/(1 + (erf0[i] / K_WL)**n_WL)) + (k_WC/(1+(clvp0[i] / K_WC )**n_WC))) * (ki_W + ((wusp0[i] / K_WW)**n_WW / (1+(wusp0[i] / K_WW)**n_WW)))  - b_Wr * wusr0[i]

                wusp[i] = k_Wp * wusr0[i] - b_Ww * wusp0[i]
                    + D_Ww * (wusp0[i+1, j] - 2 * wusp0[i, j] + wusp0[i-1, j])  +  D_Ww * (wusp0[i, j+1] - 2 * wusp0[i, j] + wusp0[i, j-1])

                clvr[i] = (k_Cr / (1+(erf0[i]/K_CL)**n_CL)) * (a_Cc + ((wusp0[i]/K_CW)**n_CW / (1 + (wusp0[i]/K_CW)**n_CW))) * (1/(1+(ham0[i]/K_CH)**n_CH))  - b_Cr * clvr0[i]

                clvp[i] = k_Cp * clvr0[i] - b_Cr * clvp0[i]
                    + D_Cc * (clvp0[i+1, j] - 2 * clvp0[i, j] + clvp0[i-1, j]) + D_Cc * (clvp0[i, j+1] - 2 * clvp0[i, j] + clvp0[i, j-1])

                erf[i] = k_Ll - b_Ll * erf0[i]  +
                    D_Ll * (erf0[i+1, j] - 2 * erf0[i, j] + erf0[i-1, j])  +  D_Ll * (erf0[i, j+1] - 2 * erf0[i, j] + erf0[i, j-1])
                """



                """
                wusr[i, j] = k_Wr * ((k_WL/(1 + (erf0[i, j] / K_WL)**n_WL)) + (k_WC/(1+(clvp0[i, j] / K_WC )**n_WC))) * (ki_W + ((wusp0[i, j] / K_WW)**n_WW / (1+(wusp0[i, j] / K_WW)**n_WW)))  - b_Wr * wusr0[i, j]

                wusp[i, j] = k_Wp * wusr0[i, j] - b_Ww * wusp0[i, j]
                    + D_Ww * (wusp0[i+1, j] - 2 * wusp0[i, j] + wusp0[i-1, j])  +  D_Ww * (wusp0[i, j+1] - 2 * wusp0[i, j] + wusp0[i, j-1])

                clvr[i, j] = (k_Cr / (1+(erf0[i, j]/K_CL)**n_CL)) * (a_Cc + ((wusp0[i, j]/K_CW)**n_CW / (1 + (wusp0[i, j]/K_CW)**n_CW))) * (1/(1+(ham0[i, j]/K_CH)**n_CH))  - b_Cr * clvr0[i, j]

                clvp[i, j] = k_Cp * clvr0[i, j] - b_Cr * clvp0[i, j]
                    + D_Cc * (clvp0[i+1, j] - 2 * clvp0[i, j] + clvp0[i-1, j]) + D_Cc * (clvp0[i, j+1] - 2 * clvp0[i, j] + clvp0[i, j-1])

                erf[i, j] = k_Ll - b_Ll * erf0[i, j]  +
                    D_Ll * (erf0[i+1, j] - 2 * erf0[i, j] + erf0[i-1, j])  +  D_Ll * (erf0[i, j+1] - 2 * erf0[i, j] + erf0[i, j-1])
                """

        # 境界条件を設定

        wusr, wusp, clvr, clvp, erf, ham =\
          boundary_condition(wusr), boundary_condition(wusp), boundary_condition(clvr),\
          boundary_condition(clvp), boundary_condition(erf), boundary_condition(ham)

        if n % result_interval == 0:
            print('Iteration=', n)


            print(wusr0)
            print(wusp0)
            print(clvr0)
            print(clvp0)
            print(erf0)

            if plsv_wusr == 1:
                plot(x, y, wusr, n, wusr_str, c_wusr, ID)
            if plsv_wusp == 1:
                plot(x, y, wusp, n, wusp_str, c_wusp, ID)
            if plsv_clvr == 1:
                plot(x, y, clvr, n, clvr_str, c_clvr, ID)
            if plsv_clvp == 1:
                plot(x, y, clvp, n, clvp_str, c_clvp, ID)
            if plsv_erf == 1:
                plot(x, y, erf, n, erf_str, c_erf, ID)
            if plsv_ham == 1:
                plot(x, y, ham, n, ham_str, c_ham, ID)

            wusr, wusp, clvr, clvp, erf, ham = wusr.T, wusp.T, clvr.T, clvp.T, erf.T, ham.T

    return

In [ ]:
"""
def boundary_condition(a):
    a[:, 0], a[:, -1], a[0, :], a[-1, :] = 0, 0, 0, 0  # left, right, up. bottom

    return a
"""

'\ndef boundary_condition(a):\n    a[:, 0], a[:, -1], a[0, :], a[-1, :] = 0, 0, 0, 0  # left, right, up. bottom\n\n    return a\n'

In [ ]:
# Parameter
points_number = 5
cutoff = 55



In [ ]:
# process

wusr, wusp, clvr, clvp, erf, ham = initial_field(points_number)
print(wusr)


neib_mx = get_neib_mx(cutoff, points_crd)
print(neib_mx)

[0. 0. 0. 0. 0.]
[[0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]]
